In [2]:
import pandas as pd
import pymongo
import os
from pymongo import MongoClient
import pprint

In [3]:
client = MongoClient(
    host = "127.0.0.1",
    port = 27017,
    username = "antoine",
    password = "pela"
)

print(client.list_database_names())

['admin', 'config', 'local', 'reco_movies']


In [4]:
db = client['reco_movies']

In [5]:
# Lister et afficher les collections
collections = db.list_collection_names()
print("Collections dans la base de données :")
for collection in collections:
    print(collection)

Collections dans la base de données :
movies
links
ratings


### CHARGEMENT DES DONNEES DANS NOS COLLECTIONS

In [7]:
# CREATION COLLECTION MOVIES
collection_movies = db['movies']

movies = pd.read_csv('/home/antoine/Ml_Ops_Movies_Reco/src/data/data/raw/movies.csv')

# Split sur les pipes
movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))
# Extraction de l'année et mise à jour du titre
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')[0]
movies['title'] = movies['title'].str.replace(r' \(\d{4}\)', '', regex=True)
movies.ffill(inplace= True)
movies.head()


,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [8]:
records_movies =movies.to_dict(orient="records")

collection_movies.insert_many(records_movies)

InsertManyResult([ObjectId('671b43809ec83baa619d7db0'), ObjectId('671b43809ec83baa619d7db1'), ObjectId('671b43809ec83baa619d7db2'), ObjectId('671b43809ec83baa619d7db3'), ObjectId('671b43809ec83baa619d7db4'), ObjectId('671b43809ec83baa619d7db5'), ObjectId('671b43809ec83baa619d7db6'), ObjectId('671b43809ec83baa619d7db7'), ObjectId('671b43809ec83baa619d7db8'), ObjectId('671b43809ec83baa619d7db9'), ObjectId('671b43809ec83baa619d7dba'), ObjectId('671b43809ec83baa619d7dbb'), ObjectId('671b43809ec83baa619d7dbc'), ObjectId('671b43809ec83baa619d7dbd'), ObjectId('671b43809ec83baa619d7dbe'), ObjectId('671b43809ec83baa619d7dbf'), ObjectId('671b43809ec83baa619d7dc0'), ObjectId('671b43809ec83baa619d7dc1'), ObjectId('671b43809ec83baa619d7dc2'), ObjectId('671b43809ec83baa619d7dc3'), ObjectId('671b43809ec83baa619d7dc4'), ObjectId('671b43809ec83baa619d7dc5'), ObjectId('671b43809ec83baa619d7dc6'), ObjectId('671b43809ec83baa619d7dc7'), ObjectId('671b43809ec83baa619d7dc8'), ObjectId('671b43809ec83baa619d7d

In [ ]:
# CREATION COLLECTION RATINGS
collection_ratings = db['ratings']

ratings = pd.read_csv('/home/antoine/Ml_Ops_Movies_Reco/src/data/data/raw/ratings.csv')

ratings = ratings.drop('timestamp', axis = 1)

records_ratings = ratings.to_dict(orient="records")

collection_ratings.insert_many(records_ratings)

In [15]:
# CREATION COLLECTION LINKS
collection_links = db['links']

links = pd.read_csv('/home/antoine/Ml_Ops_Movies_Reco/app/fastapi/app/raw/links2.csv', index_col=0)

links = links.drop('tmdbId', axis = 1)

records_links = links.to_dict(orient="records")

collection_links.insert_many(records_links)

InsertManyResult([ObjectId('671b453b9ec83baa619de83e'), ObjectId('671b453b9ec83baa619de83f'), ObjectId('671b453b9ec83baa619de840'), ObjectId('671b453b9ec83baa619de841'), ObjectId('671b453b9ec83baa619de842'), ObjectId('671b453b9ec83baa619de843'), ObjectId('671b453b9ec83baa619de844'), ObjectId('671b453b9ec83baa619de845'), ObjectId('671b453b9ec83baa619de846'), ObjectId('671b453b9ec83baa619de847'), ObjectId('671b453b9ec83baa619de848'), ObjectId('671b453b9ec83baa619de849'), ObjectId('671b453b9ec83baa619de84a'), ObjectId('671b453b9ec83baa619de84b'), ObjectId('671b453b9ec83baa619de84c'), ObjectId('671b453b9ec83baa619de84d'), ObjectId('671b453b9ec83baa619de84e'), ObjectId('671b453b9ec83baa619de84f'), ObjectId('671b453b9ec83baa619de850'), ObjectId('671b453b9ec83baa619de851'), ObjectId('671b453b9ec83baa619de852'), ObjectId('671b453b9ec83baa619de853'), ObjectId('671b453b9ec83baa619de854'), ObjectId('671b453b9ec83baa619de855'), ObjectId('671b453b9ec83baa619de856'), ObjectId('671b453b9ec83baa619de8

### VISUALISATION DES COLLECTIONS

In [16]:
ratings = db['ratings']

pprint.pprint(ratings.find_one())

{'_id': ObjectId('671b38109c1d34836e3d573b'),
 'movieId': 2,
 'rating': 3.5,
 'timestamp': 1112486027,
 'userId': 1}


In [17]:
# Compter le nombre total de documents
nombre_documents = ratings.count_documents({})
print("Nombre total de documents :", nombre_documents)

Nombre total de documents : 20000263


In [18]:
movies = db.movies

pprint.pprint(movies.find_one())

{'_id': ObjectId('671b43809ec83baa619d7db0'),
 'genres': ['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'],
 'movieId': 1,
 'title': 'Toy Story',
 'year': '1995'}


In [19]:
# Compter le nombre total de documents
nombre_movies = movies.count_documents({})
print("Nombre total de movies :", nombre_movies)

Nombre total de movies : 27278


In [20]:
links = db.links

pprint.pprint(links.find_one())

{'_id': ObjectId('671b453b9ec83baa619de83e'),
 'cover_link': 'https://m.media-amazon.com/images/M/MV5BMDU2ZWJlMjktMTRhMy00ZTA5LWEzNDgtYmNmZTEwZTViZWJkXkEyXkFqcGdeQXVyNDQ2OTk4MzI@._V1_QL75_UX190_CR0,1,190,281_.jpg',
 'imdbId': 114709,
 'movieId': 1}


### EXTRACTION DES DONNEES

In [22]:
# Récupérer le movieId maximum dans la collection links
max_movie_id_links = links.find().sort("movieId", pymongo.DESCENDING).limit(1)

# Extraire le movieId
for doc in max_movie_id_links:
    max_movie_id = doc['movieId']
    print(f"Le movieId maximum est : {max_movie_id}")

Le movieId maximum est : 131262
